In [7]:

import numpy as np
import math as m
from scipy.optimize import minimize
import sys
import platform
import os
import pandas as pd
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt
import camb
import pandas as pd



import sys
sys.path.append("../")
from pyACC.cosmology import Dc_w0waCDM

In [8]:
file_path = 'lcparam_full_long.txt' #'lcparam_full_long.txt' #'lcparam_DS17f.txt'

with open(file_path, 'r') as file:
    header_line = file.readline().strip()
    header = header_line.lstrip('#').strip()

header_words = header.split()
header_words.pop()
header = ' '.join(header_words)

data = pd.read_csv(file_path, sep='\s+', skiprows=1, header=None)
data.columns = header.split()

z_data = data['zcmb'].values
mb = data['mb'].values #B-band peak magnitude
dmb = data['dmb'].values


In [9]:
# Definition of the Hubble function and set the cosmology
def hubble_function(z,Omega_m,Omega_r,w0,wa,H0):
    DE_contribution=(1+z)**(3*(1+w0+wa))*np.exp(-3*wa*z/(1+z))
    return H0*np.sqrt(Omega_m*(1+z)**3+Omega_r*(1+z)**4+(1-Omega_m-Omega_r)*DE_contribution)

In [10]:
# Extract relevant columns
d = data['mb'].values
dmb = data['dmb'].values

# Compute the covariance matrix
Sigma = np.diag(dmb**2)
Sigma_inv = np.linalg.inv(Sigma)

#print(data.head())
print("Covariance Matrix (Sigma):")
print(Sigma)
print("Inverse Covariance Matrix (Sigma_inv):")
print(Sigma_inv)

def ln_likelihood(params, data, cov_inv):
    Omega_mat, w0, wa = params #I don't put H0 because depends on the M (absolute magnitude) , Omega_rad, Omega_k, Omega_L
    H0 = 68.
    #print(Omega_mat, Omega_rad, Omega_k, Omega_L, w0, wa)
    # Check if the parameters are within the physical range
    if Omega_mat < 0: #or Omega_rad or Omega_k or Omega_L
        #print("Negative parameter")
        return -np.inf
    #else:
        #print("Positive parameter")
    if w0 < -3 or w0 > 3:
        print("w0 out of range")
        return -np.inf
    if wa < -3 or wa > 3:
        print("wa out of range")
        return -np.inf
    z = data['zcmb'].values
    d = data['mb'].values 
    mu = []
    for z_prime in z:
        mu.append(5*np.log10(Dc_w0waCDM(hubble_function, z_prime, Omega_mat, 0., w0, wa, H0)*(1+z_prime)) + 25) #it is equivalent to get_mu(z, H0, Om0, w0, wa)
    diff = d - mu
    term1 = -0.5 * diff @ cov_inv @ diff # @ is the matrix multiplication
    term2 = 0.5 * ((np.ones_like(diff) @ cov_inv @ diff) ** 2) / (np.ones_like(diff) @ cov_inv @ np.ones_like(diff))
    #print(term1, term2, term1+term2)
    return term1 + term2
def log_prior(params):
    Omega_mat, w0, wa = params #, Omega_rad, Omega_k, Omega_L,
    if not (0.1 < Omega_mat < 0.5):
        return -np.inf
    '''
    if not (0. < Omega_rad < 0.01):
        return -np.inf
    if not (0. < Omega_k < 0.001):
        return -np.inf
    if not (0.3 < Omega_L < 0.9):
        return -np.inf
    '''
    if not (-3. < w0 < 3.):
        return -np.inf
    if not (-3. < wa < 3.):
        return -np.inf
    
    return 0.0

def log_posterior(params, data, cov_inv):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + ln_likelihood(params, data, cov_inv)

# Initialize the MCMC sampler
initial = np.array([0.3, -1., 0]) #, 0.00005, 0., 0.7
ndim = len(initial)
nwalkers = 500
pos = initial + 1e-4 * np.random.randn(nwalkers, ndim) # provare con 1e-1

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(data, Sigma_inv))

# Run the MCMC sampler
nsteps = 2000
sampler.run_mcmc(pos, nsteps, progress=True)

# Get the samples
samples = sampler.get_chain(discard=300, thin=10, flat=True)

Covariance Matrix (Sigma):
[[0.0158886  0.         0.         ... 0.         0.         0.        ]
 [0.         0.01384152 0.         ... 0.         0.         0.        ]
 [0.         0.         0.04227136 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.01365392 0.         0.        ]
 [0.         0.         0.         ... 0.         0.02394756 0.        ]
 [0.         0.         0.         ... 0.         0.         0.02168256]]
Inverse Covariance Matrix (Sigma_inv):
[[62.93819737  0.          0.         ...  0.          0.
   0.        ]
 [ 0.         72.24638764  0.         ...  0.          0.
   0.        ]
 [ 0.          0.         23.65667913 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... 73.2390271   0.
   0.        ]
 [ 0.          0.          0.         ...  0.         41.75790334
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  46.12001003]]


  0%|          | 6/2000 [03:23<18:32:06, 33.46s/it]Traceback (most recent call last):
  File "/Users/marigalla/anaconda3/envs/acc-env/lib/python3.9/site-packages/emcee/ensemble.py", line 640, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/var/folders/zj/ynn451w51tl8mfxzn0g716qr0000gn/T/ipykernel_66220/4012719553.py", line 64, in log_posterior
    return lp + ln_likelihood(params, data, cov_inv)
  File "/var/folders/zj/ynn451w51tl8mfxzn0g716qr0000gn/T/ipykernel_66220/4012719553.py", line 35, in ln_likelihood
    mu.append(5*np.log10(Dc_w0waCDM(hubble_function, z_prime, Omega_mat, 0., w0, wa, H0)*(1+z_prime)) + 25) #it is equivalent to get_mu(z, H0, Om0, w0, wa)
  File "/Users/marigalla/AstroCosmoComputational/Students/MariaLuisaGallavotti/Exam/../pyACC/cosmology/distances.py", line 32, in Dc_w0waCDM
    return op.integrate([(0,z)])
  File "/Users/marigalla/AstroCosmoComputational/Students/MariaLuisaGallavotti/Exam/../pyACC/calculus/operations.py", line 37, in integ

emcee: Exception while calling your likelihood function:
  params: [ 0.30372573 -1.00081517 -0.00405324]
  args: (        name     zcmb     zhel   dz        mb      dmb  x1  dx1  color  \
0     03D1au  0.50309  0.50309  0.0  22.93445  0.12605   0    0      0   
1     03D1ax  0.49480  0.49480  0.0  22.88020  0.11765   0    0      0   
2     03D1co  0.67767  0.67767  0.0  24.03770  0.20560   0    0      0   
3     03D1ew  0.86650  0.86650  0.0  24.34685  0.17385   0    0      0   
4     03D1fq  0.79857  0.79857  0.0  24.36050  0.17435   0    0      0   
...      ...      ...      ...  ...       ...      ...  ..  ...    ...   
1017  580276  0.33848  0.33848  0.0  21.88335  0.13905   0    0      0   
1018  590031  0.37860  0.37860  0.0  22.21555  0.14630   0    0      0   
1019  590194  0.08960  0.08960  0.0  18.72405  0.11685   0    0      0   
1020  520016  0.45070  0.45070  0.0  22.65860  0.15475   0    0      0   
1021  380199  0.46040  0.46040  0.0  22.80110  0.14725   0    0      0  

KeyboardInterrupt: 